BIBLIOTECAS

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df_2_v = pd.read_csv('VENDAS_CLIENTE_VENDEDOR_CIDADE.csv')

VERIFICANDO DATASET

In [ ]:
df_2_v.head(10)

In [ ]:
df_2_v['estado'].unique()

In [ ]:
df_2_v.info()

Verificando as colunas com linhas em branco

In [ ]:
df_2_v.isnull().sum()

Substituindo nas colunas 'estado', 'latitude' e 'longitude' valores 'NaN' por 'Não encontrado'

In [ ]:
df_2_v['estado'] = df_2_v['estado'].fillna("Não Encontrado")

In [ ]:
df_2_v['latitude'] = df_2_v['latitude'].fillna("Não Econtrado")

In [ ]:
df_2_v['longitude'] = df_2_v['longitude'].fillna("Não Econtrado")

Transformando a coluna de DIA em uma coluna formato de data

In [ ]:
df_2_v['DIA'] = pd.to_datetime(df_2_v['DIA'], errors='coerce')

agrupando as informações para não sofrer impacto das duplicadas

In [ ]:
df_pedidos = df_2_v.groupby("pedido", as_index=False).agg({
    "faturamento": "first",
    "vendedor": "first",
    "cliente": "first",
    "estado": "first",
    "latitude": "first",
    "longitude": "first"
})

1. Faturamento, ticket_medido e total de pedidos por clientes

In [ ]:
cliente_stats = (
    df_pedidos.groupby("cliente")
    .agg(
        faturamento_total=("faturamento", "sum"),
        ticket_medio=("faturamento", "mean")
    )
    .reset_index()
    .sort_values(by="faturamento_total", ascending=False)
)

In [ ]:
# Criar tabela estilizada com Plotly
cliente_stats = cliente_stats[cliente_stats['cliente'] != "Não Encontrado"]
fig = go.Figure(
    data=[go.Table(
        header=dict(
            values=["<b>Cliente</b>", "<b>Faturamento (R$)</b>", "<b>Ticket Médio</b>"],
            fill_color="royalblue",
            font=dict(color="white", size=14),
            align="center"
        ),
        cells=dict(
            values=[
                cliente_stats['cliente'],
                cliente_stats['faturamento_total'],
                cliente_stats['ticket_medio']
            ],
            fill_color=[["lavender", "white"] * (len(df_faturamento)//2 + 1)],
            align="center",
            font=dict(size=12),
            format=["", ",.2f", ",d"]  # duas casas para faturamento e inteiro p/ pedidos
        )
    )]
)

# Ajustar layout
fig.update_layout(
    title="Faturamento, ticket médio e total de pedidos por cliente",
    title_x=0.5,
    width=1200,
    height=800
)

fig.show()

2. Tota de faturamento, ticket médio e total de pedidos por vendedor

In [ ]:
vendedor_stats = df_pedidos.groupby("vendedor").agg(
    faturamento_total=("faturamento", "sum"),
    total_pedidos=("pedido", "count"),
    ticket_medio=("faturamento", "mean")
).reset_index().sort_values(by='faturamento_total', ascending=False)

In [ ]:
# Criar tabela estilizada com Plotly
vendedor_stats = vendedor_stats[vendedor_stats['vendedor'] != "Não Encontrado"]
fig = go.Figure(
    data=[go.Table(
        header=dict(
            values=["<b>Vendedor</b>", "<b>Faturamento (R$)</b>", "<b>Total de Pedidos</b>", "<b>Ticket Médio</b>"],
            fill_color="royalblue",
            font=dict(color="white", size=14),
            align="center"
        ),
        cells=dict(
            values=[
                vendedor_stats['vendedor'],
                vendedor_stats['faturamento_total'],
                vendedor_stats['total_pedidos'],
                vendedor_stats['ticket_medio']
            ],
            fill_color=[["lavender", "white"] * (len(df_faturamento)//2 + 1)],
            align="center",
            font=dict(size=12),
            format=["", ",.2f", ",d"]  # duas casas para faturamento e inteiro p/ pedidos
        )
    )]
)

# Ajustar layout
fig.update_layout(
    title="Faturamento, ticket médio e total de pedidos por Vendedor",
    title_x=0.5,
    width=1200,
    height=800
)

fig.show()

3. Faturamento e total de pedidos por estado

In [ ]:
estado_stats = df_pedidos.groupby("estado").agg(
    faturamento_total=("faturamento", "sum"),
    total_pedidos=("pedido", "count"),
    ticket_medio=("faturamento", "mean")
).reset_index().sort_values(by='faturamento_total', ascending=False)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Remover estados inválidos
estado_stats = estado_stats[estado_stats['estado'] != "Não Encontrado"]

# Selecionar os top 10 estados por faturamento
top_estados = estado_stats.sort_values(by='faturamento_total', ascending=False).head(5)

# Ordenar para o gráfico horizontal (de baixo pra cima)
top_estados = top_estados.sort_values(by='faturamento_total', ascending=False)

# Normalizar para mapa de cores
norm = plt.Normalize(top_estados['faturamento_total'].min(), top_estados['faturamento_total'].max())
colors = plt.cm.viridis(norm(top_estados['faturamento_total']))  # ou qualquer outro colormap

# Função para formatar valores em mil ou milhões
def format_milhar_milhao(valor):
    if valor >= 1_000_000:
        return f'{valor / 1_000_000:.1f} mi'
    else:
        return f'{valor / 1_000:.1f} mil'

# Plot
plt.figure(figsize=(12, 7))
ax = sns.barplot(
    x='faturamento_total',
    y='estado',
    data=top_estados,
    palette=colors
)

# Adicionar rótulos nas barras
for i, (valor, estado) in enumerate(zip(top_estados['faturamento_total'], top_estados['estado'])):
    ax.text(
        valor + 0.02 * top_estados['faturamento_total'].max(),  # deslocamento horizontal
        i,  # posição vertical
        format_milhar_milhao(valor),
        va='center',
        fontsize=10,
        color='black'
    )

# Ajustes de layout
plt.title('Top 10 Estados por Faturamento', fontsize=16)
plt.xlabel('Faturamento')
plt.ylabel('Estado')
plt.grid(axis='x', linestyle='--', alpha=0.4)

# Aumentar o limite do eixo X para dar espaço aos rótulos
plt.xlim(0, top_estados['faturamento_total'].max() * 1.15)

plt.tight_layout()
plt.show()


Como pode ser visto o estado com maior faturamento foi o estado de São Paulo com quase 10 milhões em vendas

4. Mapa de faturamento por região

In [ ]:
import plotly.express as px

# Consolidar faturamento único por pedido
df_pedidos = df_2_v.groupby("pedido", as_index=False).agg({
    "faturamento": "first",
    "latitude": "first",
    "longitude": "first"
})

# Mapa com bolas quentes
mapa_bolhas = px.scatter_mapbox(
    df_pedidos,
    lat="latitude",
    lon="longitude",
    size="faturamento",              
    color="faturamento",             
    color_continuous_scale="OrRd",   # laranja → vermelho
    hover_data={"faturamento": True},
    zoom=3,
    center={"lat": -15.8, "lon": -47.9},  
    height=700,
    width=1000,
    title="Faturamento por Região"
)

# Ajustar escala de tamanho
mapa_bolhas.update_traces(
    marker=dict(
        sizemode="area",   # mantém proporção pela área da bolha
        sizeref=2.*max(df_pedidos["faturamento"])/(40.**2),  # controla bolha máxima
        sizemin=3          # tamanho mínimo
    )
)

# Estilo do mapa
mapa_bolhas.update_layout(mapbox_style="open-street-map")

mapa_bolhas.show()


Como pode ser visto a região sudeste e centro oeste e suldeste do pais foram as que lideraram em vendas